# MUDAB2G1 - Grupo 3 - TFM: Modelo de predicción de precios de acciones

Este notebook pertenece al Grupo 3 del MUDAB2G1. Cuyos miembros de equipo son:

*   Carrillo Ng, Sebastián Elías
*   Cifuentes Guzmán, Yaros Josué
*   Donoso Muñoz, Javier Esteban

## Objetivo del Notebook: construcción de datasets

El objetivo de este notebook es preparar dos datasets relevantes para la creación de nuestro modelo de predicción de precios de las acciones del mercado bursátil. Para ello utilizaremos dos pipelines (i) extracción de precios de acciones y (ii) extracción de noticias. En cada una de las secciones se detalla con mayor profundidad la metodología y razonamiento planteado.

## Pipeline de Extracción de Precios de Acciones

Esta sección tiene como objetivo recopilar los precios históricos de las acciones correspondientes a las empresas que serán objeto de estudio en esta investigación. Para ello, se emplea la librería yfinance, la cual permite acceder de forma programática a datos financieros provenientes de Yahoo Finance.

Se procederá a enriquecer el conjunto de datos con una serie de variables derivadas que permiten caracterizar aspectos esenciales del comportamiento del activo.

En resumen, el pipeline propuesto sigue el siguiente orden:

Recolección de precios de acciones → Enriquecimiento de features → Exportación a CSV

### Preparación del entorno

Se importan todas las librerías necesarias para la obtención de datos

In [ ]:
import yfinance as yf
import datetime
from datetime import datetime, date
import pandas as pd

### Obtención de precios

Se utiliza el parámetro del nombre de la empresa en la Bolsa de Valores americana para poder obtener un dataset financiero

In [ ]:
# Parámetros
stock = 'TSLA'
start = datetime.datetime(2018, 12, 1)
end = datetime.datetime(2025, 1, 31)

# Descargar datos
data = yf.download(stock, start=start, end=end, actions=True)
data = data.reset_index()
data.head(1)

In [ ]:
# Aplanar columnas si tienen MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]

# Verificar nombres actuales
print("Columnas originales:", data.columns.tolist())

# Forzar nombres limpios (orden esperado de columnas)
df = data.copy()
df.columns = ['date', 'close', 'dividends', 'high', 'low', 'open', 'stock_splits', 'volume']

# Reordenar columnas en el orden deseado
desired_order = ['date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits']
df = df[desired_order]

# Resultado final
print("✅ DataFrame limpio:")
df.head()

### Ingeniería de features

Se crean una serie de variables adicionales a partir del precio histórico que capturan diferentes dimensiones del comportamiento bursátil. Entre ellas se incluyen medidas de tendencia como la media móvil exponencial (EMA), valores pasados del precio que introducen memoria temporal, retornos porcentuales diarios y semanales que reflejan la rentabilidad reciente, y métricas de volatilidad como la desviación estándar móvil y el rango de precios intradiarios. Estas variables permiten al modelo incorporar señales cuantitativas clave del mercado en sus predicciones.

In [ ]:
# Calcular media móvil exponencial
df['ema'] = df['close'].ewm(span=15, adjust=False).mean()

# Crear variables derivadas
df['close_lag1'] = df['close'].shift(1)
df['close_lag2'] = df['close'].shift(2)
df['return_1d'] = df['close'].pct_change(1)
df['return_7d'] = df['close'].pct_change(7)
df['volatility_7d'] = df['close'].rolling(window=7).std()
df['price_range'] = df['high'] - df['low']
df['price_range_7d'] = df['price_range'].rolling(window=7).mean()

# Limpiar valores nulos
df = df.sort_values('date').reset_index(drop=True)
# Guardar resultado
df.to_csv(f"{stock}_stock_features.csv", index=False)
df.head()

## Pipeline de Extracción de Noticias

Esta sección del notebook tiene como objetivo recopilar noticias financieras relevantes, extraer su contenido y analizar su sentimiento para integrarlos como variables explicativas en un modelo de predicción del precio de acciones.

A través de diferentes secciones, se emplean herramientas como GNews, técnicas de web scraping, y modelos de análisis de sentimiento como TextBlob, Vader y FinBERT. El pipeline está diseñado para automatizar el proceso desde la recolección de noticias hasta la generación de variables numéricas que representen el tono de los artículos.

En resumen, el pipeline propuesto sigue el siguiente orden:

Recolección de noticias → Limpieza de URLs → Extracción de texto → Análisis de sentimiento → Exportación a CSV

### Preparar el entorno

Se importan todas las librerías que se utilizaran en esta sección del proyecto

In [ ]:
!pip install swifter
!pip install scipy
!pip install tqdm
!pip install requests
!pip install beautifulsoup4
!pip install newspaper3k
!pip install selenium
!pip install nltk
!pip install gnews
!pip install googlenewsdecoder
!pip install textblob
!pip install transformers
!pip install vaderSentiment
!pip install torch

import os
import re
import time
import urllib.parse
from datetime import datetime, timedelta

os.environ["TRANSFORMERS_NO_TF"] = "1"

# Manejo de datos
import numpy as np
import pandas as pd
from scipy.special import softmax
from tqdm import tqdm
from tqdm.notebook import tqdm as notebook_tqdm
import swifter

# Scrappeo y parseo
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Herramientas NLP
import nltk
from gnews import GNews
from googlenewsdecoder import new_decoderv1
from textblob import TextBlob, download_corpora
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch
nltk.download('punkt')

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata (572 bytes)
Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found exis

ImportError: lxml.html.clean module is now a separate project lxml_html_clean.
Install lxml[html_clean] or lxml_html_clean directly.

### Obtención de noticias (GNews)

En esta sección se encuentra el código para recorrer la web de GNews y conseguir las noticias por año de alguna empresa en particular (en este caso, para Tesla). A diferencia de las APIs de pago, este código realiza una búsqueda de acuerdo a la densidad de lo encontrado en internet sobre ese término. Parte de la premisa es que este código limita las búsquedas a un máximo de 100 documentos por consulta. Para evitar bloqueos y asegurar la obtención de los datos, introduce pausas entre cada día e itera sobre ellos de forma controlada.

In [ ]:
def fx_timestamp():
    from datetime import datetime
    """
    Va a devolver un sello temporal para tenerlo en caso sea necesario
    """
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-5]
    return timestamp

In [ ]:
def split_date_range_and_process(keyword, start_date, end_date, sparseness):
    """
    Divide un rango de fechas en intervalos más pequeños según el nivel de dispersión y ejecuta una función en cada intervalo.

    Parámetros:
        start_date (str): Fecha de inicio en formato 'YYYY-MM-DD'.
        end_date (str): Fecha de fin en formato 'YYYY-MM-DD'.
        sparseness (str): Nivel de dispersión ('very sparse', 'sparse', 'medium', 'dense').
        function (callable): Función a ejecutar en cada intervalo. Debe aceptar start_date y end_date como argumentos.

    Retorna:
        pd.DataFrame: Resultados combinados de la función ejecutada en todos los intervalos.
    """
    # Convertir las fechas
    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")

    # Inicializar las variables
    current_start_date = start_date_obj
    combined_df = pd.DataFrame()

    # Hacer un loop por las fechas
    while current_start_date < end_date_obj:

        # Determinar el sparseness en base a lo que creemos que vamos a encontrar la data.

        if sparseness == 1:                           # "very sparse" = yearly
            increment = timedelta(days=365)
        elif sparseness == 2:                         # "sparse" = monthly
            increment = timedelta(days=30)
        elif sparseness == 3:                         # "medium" = weekly
            increment = timedelta(days=7)
        else:  # 4                                   # "dense" = daily
            increment = timedelta(days=1)


        current_end_date = current_start_date + increment
        if current_end_date > end_date_obj:
            current_end_date = end_date_obj

        start_date = current_start_date.strftime("%Y-%m-%d")
        end_date = current_end_date.strftime("%Y-%m-%d")

        # Función para sacar las noticias
        df, num_records = fetch_press_googlenewsrss_daterange(keyword, start_date, end_date, language = 'en')

        # Unir los resultados al dataframe combinado
        if not df.empty:
            combined_df = pd.concat([combined_df, df], ignore_index=True)

        # Moverse al siguiente intervalo
        current_start_date = current_end_date

        print("Sparseness at " + str(current_start_date) + ": "+ str(sparseness) + ". Records = " + str(num_records))

        if num_records >= 100:
            sparseness = sparseness + 1

        if sparseness == 5:
            print("Max records per search for "+ keyword + " reached at " + str(current_end_date) + " (daily). Recommend using a more specific keyword going forward.")

    return combined_df

def fetch_press_googlenewsrss_daterange(keyword, start_date, end_date, language = 'en'):

    all_dataframes = []

    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")

    start_date_str = str(start_date_obj)
    end_date_str = str(end_date_obj)

    print(start_date_str, end_date_str)

    # Traer todas las noticias de esa fecha
    # Ponerle 5 segundos para que no sea tan rápido y no se nos bloquee el scrappeo
    time.sleep(5)

    google_news = GNews(
    language=language,
    max_results=100 # 100 es el resultado máximo que puede salir en la búsqueda
    )

    google_news.start_date = (start_date_obj.year, start_date_obj.month, start_date_obj.day)
    google_news.end_date = (end_date_obj.year, end_date_obj.month, end_date_obj.day)

    news_json = google_news.get_news(keyword)

    df = pd.json_normalize(
        news_json,
        sep='_',
        errors='ignore'
    )

    if not df.empty:
        # Renombrar las columnas para un mejor entendimiento
        df = df.rename(columns={
            'title': 'ent_name',
            'description': 'ent_summary',
            'published date': 'ent_start',
            'publisher_href': 'contentOriginUrl',
            'publisher_title': 'ent_press_name'
        })

        df['ent_start'] = pd.to_datetime(df['ent_start'], format='%a, %d %b %Y %H:%M:%S %Z', errors='coerce')
        df['ent_start'] = df['ent_start'].dt.strftime('%Y-%m-%d')
        df['ent_print_type'] = 'news'

    # Poner el folder donde se van a guardar las noticias
        df.to_csv("unprocessed_press/" + keyword + "_press_" + start_date_str + " to "+ end_date_str + ".csv", index=False)

        timestamp = fx_timestamp()
        num_records = len(df)
        print('Records for '+ keyword + ': ' + str(num_records) + ' fetched at '+ timestamp )

        return df, num_records

In [ ]:
keyword = 'Tesla, inc'
start_date = '2024-01-01' #2017-08-22 formato
end_date = '2024-12-31'
sparseness = 3

df_press = split_date_range_and_process(keyword, start_date, end_date, sparseness)
df_press

In [ ]:
# Este bloque de código sirve para juntar todos los archivos en uno y poder empezar a extraer el URL original

folder_path = "unprocessed_press"  # carpeta donde están los CSVs descargados
subject = "Tesla, inc"  # usar el nombre que se usó como keyword

# Listar todos los archivos CSV relevantes para el subject
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv") and subject in f]

# Leer y juntar todos los archivos
df_list = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatenar todos los DataFrames
df_combined = pd.concat(df_list, ignore_index=True)

print(f"Archivos combinados: {len(csv_files)}")
print(f"Total de noticias combinadas: {len(df_combined)}")

### URL original

El método previo para obtener las noticias nos da un URL en formato RSS (Really Simple Syndication). Esto quiere decir que necesitamos ir más allá para poder realizar la extracción del texto y posterior análisis de sentimiento. Cabe precisar que, este método toma varios segundos por fila y, por ende, se opta por trabajarlo por batch de 1000 filas lo cual dura aproximadamente 1 hora y 30 minutos por año.

In [ ]:
# Borrar duplicados en base al 'title' y 'publish_date'

df = df_combined
df = df.drop_duplicates(subset=["title", "publish_date"])
df = df.reset_index(drop=True)

print(f"Duplicados borrados. Tamaño del DF: {df.shape}")

In [ ]:
# Funcion para decodificar el RSS
def get_original_url(source_url):
    interval_time = 5
    try:
        decoded_url = new_decoderv1(source_url, interval=interval_time)
        if decoded_url.get("status"):
            return decoded_url["decoded_url"]
        else:
            return source_url
    except Exception as e:
        print(f"⚠️ Error occurred with {source_url}: {e}")
        return None

# Cargar data
df = pd.read_csv("tesla_inc_unprocessed_press_filtered.csv")
df['ent_start'] = pd.to_datetime(df['ent_start'], errors='coerce')

# Filtrar por 2019
df_2019 = df[df['ent_start'].dt.year == 2019].copy()
print(f"📅 Processing {len(df_2019)} rows from 2019")

# Crear el folder donde se van a guardar los batches
output_folder = "batches_2021_tesla"
os.makedirs(output_folder, exist_ok=True)

# El proceso toma mucho tiempo, por ende, se realiza el procesamiento en batches de 1000.
batch_size = 1000
num_batches = (len(df_2019) // batch_size) + 1

# Procesamiento
for batch_num in range(num_batches):
    start = batch_num * batch_size
    end = min((batch_num + 1) * batch_size, len(df_2019))
    batch_df = df_2019.iloc[start:end].copy()

    print(f"🔄 Batch {batch_num + 1}/{num_batches}: Rows {start} to {end}")

    # Utilizamiento de swifter para que sea más rápido (procesamiento paralelo de manera automática cuando es posible)
    tqdm.pandas(desc=f"Batch {batch_num + 1} decoding")
    batch_df['expanded_url'] = batch_df['url'].swifter.apply(get_original_url)

    # Guardar al CSV
    output_file = f"{output_folder}/2019_tesla_batch_{batch_num + 1}.csv"
    batch_df.to_csv(output_file, index=False)
    print(f"✅ Saved batch {batch_num + 1} to {output_file}\n")

📅 Processing 1216 rows from 2019
🔄 Batch 1/2: Rows 0 to 1000


Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

✅ Saved batch 1 to batches_2021_apple/2019_apple_batch_1.csv

🔄 Batch 2/2: Rows 1000 to 1216


Pandas Apply:   0%|          | 0/216 [00:00<?, ?it/s]

✅ Saved batch 2 to batches_2021_apple/2019_apple_batch_2.csv



In [ ]:
# Este bloque del código sirve para juntar todos los archivos en uno y poder realizar el análisis de sentimiento

FOLDER_PATH = "batches_2021_tesla"
OUTPUT_FILE = "combined_batches.csv"

all_files = list(Path(FOLDER_PATH).glob("*.csv"))
if not all_files:
    print(f"No se encontrar archivos CSV {FOLDER_PATH}")
else:
    df_list = []
    for file in all_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Fallo al leer {file.name}: {e}")

    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        combined_df.to_csv(OUTPUT_FILE, index=False)
        print(f"Combinar {len(df_list)} archivos en '{OUTPUT_FILE}'")
    else:
        print("No hay CSV válidos para combinar")

✅ Combined 20 files into 'combined_batches.csv'


### Resumen de la noticia

Posterior a obtener el link original de la noticia obtenida, se procede a borrar duplicados y a realizar el doble click en dicho link, con el fin de extraer un resumen que será posteriormente analizado para determinar el sentimiento asociado a la noticia. Sin embargo, se identifica un problema con algunas webs que detectan al código como si fuese un robot. Para estas noticias donde no es posible obtener el resumen de la noticia, se opta por utilizar el título de la misma.

In [ ]:
df = pd.read_csv("combined_batches.csv")

In [ ]:
df = df[['ent_name', 'ent_start', 'url', 'ent_press_name', 'expanded_url']]

In [ ]:
df = df.rename(columns={
    "ent_name": "title",
    "ent_start": "publish_date",
    "ent_press_name": "source",
})

In [ ]:
df.head(1)

,title,publish_date,url,source,expanded_url
0,Wall Street sinks as hopes wane for tariff del...,2025-04-08,https://news.google.com/rss/articles/CBMixwFBV...,marketscreener.com,https://www.marketscreener.com/quote/stock/APP...


In [ ]:
# Parámetros
OUTPUT_DIR = "summary_tesla_folder"
BATCH_SIZE = 1000

# Crear la carpeta del parámetro si es que no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Se utiliza el tqdm para poder seguir el progreso que vamos teniendo
tqdm.pandas(desc="Generando resúmenes")

# Función para obtener los resúmenes
def get_summary(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"Falló extraer el resumen {url} porque {e}")
        return None

# Procesamiento en batches
num_batches = (len(df) // BATCH_SIZE) + 1

for i in range(num_batches):
    start = i * BATCH_SIZE
    end = min((i + 1) * BATCH_SIZE, len(df))
    batch = df.iloc[start:end].copy()

    print(f"\n Procesando el batch {i+1}/{num_batches} (filas {start}–{end})...")

    batch['summary'] = batch['expanded_url'].progress_apply(get_summary)

    output_path = os.path.join(OUTPUT_DIR, f"tesla_summary_batch_{i+1}.csv")
    batch.to_csv(output_path, index=False)
    print(f"✅ Saved batch {i+1} to {output_path}")


📦 Processing batch 1/17 (rows 0–1000)...


📰 Generating Summaries:   0%|          | 0/1000 [00:00<?, ?it/s]

⚠️ Failed to extract summary from https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/Wall-Street-sinks-as-hopes-wane-for-tariff-delays-deals-49566588/ because Article `download()` failed with 403 Client Error: Forbidden for url: https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/Wall-Street-sinks-as-hopes-wane-for-tariff-delays-deals-49566588/ on URL https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/Wall-Street-sinks-as-hopes-wane-for-tariff-delays-deals-49566588/
⚠️ Failed to extract summary from https://m.uk.investing.com/news/-4021762?ampMode=1 because Article `download()` failed with 403 Client Error: Forbidden for url: https://m.uk.investing.com/news/-4021762?ampMode=1 on URL https://m.uk.investing.com/news/-4021762?ampMode=1
⚠️ Failed to extract summary from https://m.ng.investing.com/news/stock-market-news/apple-stock-hits-11mth-low-on-tariff-fears-falls-behind-microsoft-in-market-cap-1857223?ampMode=1 because Article `download()` failed with

📰 Generating Summaries:   0%|          | 0/1000 [00:00<?, ?it/s]

⚠️ Failed to extract summary from https://tech.hindustantimes.com/mobile/news/this-is-how-many-apple-iphones-the-company-will-make-in-2022-71653572203905.html because Article `download()` failed with 403 Client Error: Forbidden for url: https://tech.hindustantimes.com/mobile/news/this-is-how-many-apple-iphones-the-company-will-make-in-2022-71653572203905.html on URL https://tech.hindustantimes.com/mobile/news/this-is-how-many-apple-iphones-the-company-will-make-in-2022-71653572203905.html
⚠️ Failed to extract summary from https://www.communicationstoday.co.in/apple-encore/ because Article `download()` failed with HTTPSConnectionPool(host='www.communicationstoday.co.in', port=443): Read timed out. (read timeout=7) on URL https://www.communicationstoday.co.in/apple-encore/
⚠️ Failed to extract summary from https://siliconangle.com/2024/01/25/comply-eu-law-apple-opens-door-third-party-app-stores-europe/ because Article `download()` failed with 403 Client Error: Forbidden for url: https://

📰 Generating Summaries:   0%|          | 0/1000 [00:00<?, ?it/s]

⚠️ Failed to extract summary from https://www.reuters.com/technology/yen-tumbles-gadget-loving-japan-goes-secondhand-iphones-2022-11-08/ because Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/technology/yen-tumbles-gadget-loving-japan-goes-secondhand-iphones-2022-11-08/ on URL https://www.reuters.com/technology/yen-tumbles-gadget-loving-japan-goes-secondhand-iphones-2022-11-08/
⚠️ Failed to extract summary from https://www.bloomberg.com/news/articles/2022-11-06/risk-tone-sours-dollar-rises-on-china-covid-zero-markets-wrap because Article `download()` failed with 403 Client Error: Forbidden for url: https://www.bloomberg.com/news/articles/2022-11-06/risk-tone-sours-dollar-rises-on-china-covid-zero-markets-wrap on URL https://www.bloomberg.com/news/articles/2022-11-06/risk-tone-sours-dollar-rises-on-china-covid-zero-markets-wrap
⚠️ Failed to extract summary from https://www.bloomberg.com/opinion/articles/2022-11-10/elon-musk-makes-rto-t

📰 Generating Summaries:   0%|          | 0/1000 [00:00<?, ?it/s]

⚠️ Failed to extract summary from https://www.bloomberg.com/news/articles/2025-01-16/disney-amazon-are-in-talks-to-stream-la-fires-charity-concert because Article `download()` failed with 403 Client Error: Forbidden for url: https://www.bloomberg.com/news/articles/2025-01-16/disney-amazon-are-in-talks-to-stream-la-fires-charity-concert on URL https://www.bloomberg.com/news/articles/2025-01-16/disney-amazon-are-in-talks-to-stream-la-fires-charity-concert
⚠️ Failed to extract summary from https://www.benzinga.com/analyst-ratings/analyst-color/25/01/43021026/apple-stock-has-moved-up-4-2-since-iphone-16-launched-analyst-says-it-has-another-8-upside-as-manufacturing-cost-decline-bolsters-cupertinos-margins because Article `download()` failed with 429 Client Error: Too Many Requests for url: https://www.benzinga.com/analyst-ratings/analyst-color/25/01/43021026/apple-stock-has-moved-up-4-2-since-iphone-16-launched-analyst-says-it-has-another-8-upside-as-manufacturing-cost-decline-bolsters-cup

### Análisis de sentimientos

Con los resumenes de los textos y con los títulos de las noticias, se procede a realizar la obtención del sentimiento de los mismos. Para ello, se optaron 3 librerías para poder entender cuál era aquella que nos sería más util: Textblob, Vader y FinBert. De acuerdo con los resultados y por su mejor similitud entre Vader y FinBert, nos quedamos con el último, ya que esta librería está enfocada en textos financieros.

#### Textblob y Vader

TextBlob: Librería basada en diccionarios y reglas gramaticales, adecuada para textos generales. Retorna un valor de polaridad entre -1 y +1. Se empleó como punto de referencia o línea base.

VADER: Diseñado específicamente para textos cortos e informales, como titulares de noticias. Utiliza un enfoque basado en reglas lingüísticas para captar matices afectivos. También produce una puntuación entre -1 y +1.

In [ ]:
# Cargar base de datos
df = pd.read_csv("combined_batches_summary.csv")

# Crear carpeta de salida para la etapa 1
os.makedirs("sentiment_batches_text", exist_ok=True)

# Inicializar VADER
vader = SentimentIntensityAnalyzer()

# Definir funciones de sentimiento
def get_textblob_sentiment(text):
    if isinstance(text, str) and text.strip():
        return TextBlob(text).sentiment.polarity
    return None

def get_vader_sentiment(text):
    if isinstance(text, str) and text.strip():
        return vader.polarity_scores(text)['compound']
    return None

# Procesamiento en batches
batch_size = 1000
total_rows = len(df)

for i in range(0, total_rows, batch_size):
    end = min(i + batch_size, total_rows)
    print(f"\n📝 Processing TextBlob & VADER: Rows {i} to {end}")

    batch = df.iloc[i:end].copy()

    tqdm.pandas(desc="TextBlob title")
    batch['textblob_title_score'] = batch['title'].progress_apply(get_textblob_sentiment)

    tqdm.pandas(desc="TextBlob summary")
    batch['textblob_summary_score'] = batch['summary'].progress_apply(get_textblob_sentiment)

    tqdm.pandas(desc="VADER title")
    batch['vader_title_score'] = batch['title'].progress_apply(get_vader_sentiment)

    tqdm.pandas(desc="VADER summary")
    batch['vader_summary_score'] = batch['summary'].progress_apply(get_vader_sentiment)

    # Guardar el batch con solo TextBlob/VADER
    output_file = f"sentiment_batches_text/sentiment_batch_text_{i}_{end}.csv"
    batch.to_csv(output_file, index=False)
    print(f"✅ Saved batch to {output_file}")

NameError: name 'nltk' is not defined

#### FinBert

Modelo basado en la arquitectura BERT, entrenado específicamente con textos del ámbito financiero. Clasifica cada entrada como positiva, neutral o negativa, y permite calcular un puntaje ponderado en función de la distribución de dichas categorías. Su diseño especializado lo hace especialmente adecuado para contextos bursátiles y económicos.

In [ ]:
# Organizar carpetas
input_folder = "sentiment_batches_text"
output_folder = "sentiment_batches_full"
os.makedirs(output_folder, exist_ok=True)

# FinBERT
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
labels = ['negative', 'neutral', 'positive']

# Función para Finbert
def get_finbert_label_and_score(text):
    if not isinstance(text, str) or len(text.strip()) < 5:
        return (None, None)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = softmax(outputs.logits.numpy()[0])
    best_idx = probs.argmax()
    return (labels[best_idx], float(probs[best_idx]))

/opt/tljh/user/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Detectar el último batch realizado
existing_outputs = [
    f for f in os.listdir(output_folder)
    if f.startswith("sentiment_batch_full_") and f.endswith(".csv")
]

if existing_outputs:
    # Extraer el mayor número del nombre
    last_index = max([
        int(f.split("_")[-1].replace(".csv", ""))
        for f in existing_outputs
    ])
else:
    last_index = -1  # aún no hay nada procesado

# Procesar el siguiente batch
input_files = sorted([
    f for f in os.listdir(input_folder)
    if f.startswith("sentiment_batch_text_") and f.endswith(".csv")
])

for file in input_files:
    file_index = int(file.split("_")[-1].replace(".csv", ""))
    if file_index <= last_index:
        continue  # ya fue procesado

    input_path = os.path.join(input_folder, file)
    output_path = os.path.join(output_folder, file.replace("text", "full"))

    print(f"\n🔄 Procesando {file}")
    df = pd.read_csv(input_path)

    # FinBERT para título
    tqdm.pandas(desc="FinBERT title sentiment")
    title_result = df['title'].progress_apply(get_finbert_label_and_score)
    df['finbert_title_label'] = title_result.apply(lambda x: x[0])
    df['finbert_title_score'] = title_result.apply(lambda x: x[1])

    # FinBERT para resumen
    tqdm.pandas(desc="FinBERT summary sentiment")
    summary_result = df['summary'].progress_apply(get_finbert_label_and_score)
    df['finbert_summary_label'] = summary_result.apply(lambda x: x[0])
    df['finbert_summary_score'] = summary_result.apply(lambda x: x[1])

    # Guardar
    df.to_csv(output_path, index=False)
    print(f"✅ Guardado: {output_path}")


🔄 Processing sentiment_batch_text_22000_22753.csv


FinBERT title sentiment:   0%|          | 0/753 [00:00<?, ?it/s]

FinBERT summary sentiment:   0%|          | 0/753 [00:00<?, ?it/s]

✅ Saved to sentiment_batches_full/sentiment_batch_full_22000_22753.csv


In [ ]:
# Batches de FinBERT
folder_path = "/home/jupyter-sebas/sentiment_batches_full" # ajustar esta ruta según el entorno desde donde se ejecute el código

# Listar todos los csvs en el folder
csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

# Cargarlos y coleccionarlos en un Dataframe
df_list = []

for file in csv_files:
    path = os.path.join(folder_path, file)
    df = pd.read_csv(path)
    df_list.append(df)

# Combinarlos
df_full = pd.concat(df_list, ignore_index=True)

# Guardarlo en un csv
df_full.to_csv("tesla_sentiment_full.csv", index=False)
print(f"Se combinaron {len(csv_files)} archivos con éxito ")

✅ Done! Combined 22 files into 'tesla_sentiment_full.csv'


In [ ]:
# Cargar el DF
df = pd.read_csv("tesla_sentiment_full.csv")

# FinBERT devuelve una probabilidad y un label si es que es positivo, neutro o negativo. Por lo tanto, creamos nuestro puntaje propio (label * score)
def scale_label_with_confidence(label, score):
    label_to_weight = {
        "positive": 1,
        "neutral": 0,
        "negative": -1
    }
    if pd.isna(label) or pd.isna(score):
        return None
    return label_to_weight.get(label.lower(), 0) * score

In [ ]:
# Se obtiene el puntaje asociado al título
df['finbert_title_sentiment_score'] = df.apply(
    lambda row: scale_label_with_confidence(row['finbert_title_label'], row['finbert_title_score']),
    axis=1
)

# Se obtiene el puntaje asociado al resumen
df['finbert_summary_sentiment_score'] = df.apply(
    lambda row: scale_label_with_confidence(row['finbert_summary_label'], row['finbert_summary_score']),
    axis=1
)

In [ ]:
df.to_csv("tesla_sentiment_full_weighted.csv", index=False)
print("Se guardó con éxito el archivo csv")

✅ Saved final file with FinBERT sentiment scores to 'tesla_sentiment_full_weighted.csv'


In [ ]:
df = pd.read_csv("tesla_sentiment_full_weighted.csv")

In [ ]:
# Como se mencionó previamente, existen algunas noticias que no se pudieron procesar correctamente, debido a restricciones o a mecanismos de detección
# que identificaban la solicitud como proveniente de un bot. Por lo tanto, en los casos en que no se logró extraer el resumen, se optó por utilizar como alternativa
# el título de la noticia junto con su respectiva puntuación de sentimiento.

df['finbert_final_sentiment'] = df['finbert_summary_sentiment_score'].combine_first(
    df['finbert_title_sentiment_score']
)

In [ ]:
df.to_csv("tesla_sentiment_final.csv")

#### FinBERT embeddings


En esta sección se genera una representación numérica (embedding) del contenido textual de cada noticia utilizando el modelo preentrenado **FinBERT**, una versión de BERT adaptada específicamente al lenguaje financiero.

Cada embedding es un vector de 768 dimensiones que captura el significado semántico del texto, incluyendo tono, contexto financiero y entidades mencionadas. Estos vectores permiten representar el contenido de la noticia de forma que pueda ser utilizada como input en modelos de predicción, como LSTM.

Para cada artículo, se utiliza el **summary** si está disponible, o el **title** en su defecto, asegurando así una cobertura completa del dataset.

In [ ]:
# Cargar el archivo CSV con los datos finales de Tesla
df = pd.read_csv("tesla_final_data.csv")

In [ ]:
# Seleccionar solo las columnas relevantes para el análisis
df = df[['title', 'publish_date', 'expanded_url', 'source', 'summary', 'finbert_final_sentiment']]

In [ ]:
# Mostrar la primera fila del DataFrame como muestra
df.head(1)

,title,publish_date,expanded_url,source,summary,finbert_final_sentiment
0,Product of the Week: Advantech’s NVIDIA Jetson...,2022-03-15,https://embeddedcomputing.com/technology/ai-ma...,Embedded Computing Design,Product of the Week: Advantech’s NVIDIA Jetson...,0.861443


In [ ]:
# Configuración
company_name = "tesla"
output_folder = f"embedding_batches_{company_name}"
batch_size = 1000
os.makedirs(output_folder, exist_ok=True)

# Cargar modelo FinBERT
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")

# Determinar texto a usar
df['text_input'] = df['summary'].fillna(df['title'])

# Encontrar último batch completado
existing_batches = [f for f in os.listdir(output_folder) if f.startswith("embedding_batch_")]
if existing_batches:
    last_batch = max(int(f.split("_")[-1].split(".")[0]) for f in existing_batches)
    start_row = last_batch * batch_size
    print(f"📦 Retomando desde batch {last_batch + 1}, fila {start_row}")
else:
    last_batch = 0
    start_row = 0
    print("Iniciando desde el principio")

# Función de embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Procesar en batches
num_batches = (len(df) // batch_size) + 1
for batch_num in range(last_batch, num_batches):
    start = batch_num * batch_size
    end = min((batch_num + 1) * batch_size, len(df))
    chunk = df.iloc[start:end].copy()

    print(f"Batch {batch_num + 1}/{num_batches}: Filas {start} a {end}")

    embeddings = []
    for text in tqdm(chunk['text_input'], desc=f"Embedding batch {batch_num + 1}", leave=False):
        try:
            emb = get_embedding(text)
        except Exception as e:
            print(f"Error en texto: {e}")
            emb = [None] * model.config.hidden_size
        embeddings.append(emb)

    emb_df = pd.DataFrame(embeddings)
    result = pd.concat([chunk.reset_index(drop=True), emb_df], axis=1)

    result.to_csv(f"{output_folder}/embedding_batch_{batch_num + 1}.csv", index=False)
    print(f"Guardado: embedding_batch_{batch_num + 1}.csv")

🆕 Iniciando desde el principio
🔄 Batch 1/39: Filas 0 a 1000


✅ Guardado: embedding_batch_1.csv
🔄 Batch 2/39: Filas 1000 a 2000


✅ Guardado: embedding_batch_2.csv
🔄 Batch 3/39: Filas 2000 a 3000


✅ Guardado: embedding_batch_3.csv
🔄 Batch 4/39: Filas 3000 a 4000


✅ Guardado: embedding_batch_4.csv
🔄 Batch 5/39: Filas 4000 a 5000


🧠 Embedding batch 5:  42%|███████▌          | 418/1000 [00:15<00:22, 25.81it/s]

In [ ]:
# Ruta del folder donde están los batches de embeddings

embedding_folder = f"embedding_batches_{company_name}"
output_file = f"{company_name}_final_news.csv"

# Listar todos los CSVs que empiecen con "embedding_batch_"
embedding_files = sorted([
    f for f in os.listdir(embedding_folder)
    if f.endswith(".csv") and f.startswith("embedding_batch_")
])

# Leer y acumular los archivos
dfs = []
for file in embedding_files:
    path = os.path.join(embedding_folder, file)
    df = pd.read_csv(path)
    dfs.append(df)

# Concatenar todo
final_df = pd.concat(dfs, ignore_index=True)

# Guardar como archivo final
final_df.to_csv(output_file, index=False)
print(f"Archivo final guardado como: {output_file}")
print(f"Total de filas: {len(final_df)}")

✅ Archivo final guardado como: tesla_final_news.csv
🧾 Total de filas: 21753


#### Dataframe final

In [ ]:
# Cargar el archivo CSV con las noticias finales de Tesla y mostrar la primera fila
df = pd.read_csv("tesla_final_news_1.csv")
df.head(1)

,title,publish_date,expanded_url,source,summary,finbert_final_sentiment,text_input,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Product of the Week: Advantech’s NVIDIA Jetson...,2022-03-15,https://embeddedcomputing.com/technology/ai-ma...,Embedded Computing Design,Product of the Week: Advantech’s NVIDIA Jetson...,0.861443,Product of the Week: Advantech’s NVIDIA Jetson...,-0.076522,0.367546,0.218021,...,-0.407227,0.070884,0.000212,-0.348598,-0.482147,0.260011,0.212645,-0.593635,-0.369286,0.021706


In [ ]:
# Convertir la columna 'publish_date' a formato datetime
df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')

# Crear una lista con los nombres de las columnas que representan los embeddings (768 dimensiones)
embedding_cols = [str(i) for i in range(768)]

# Agrupar por fecha y calcular promedios
daily_aggregated_df = df.groupby('publish_date')[
    ['finbert_final_sentiment'] + embedding_cols
].mean().reset_index()

print(daily_aggregated_df.head())

  publish_date  finbert_final_sentiment         0         1         2  \
0   2019-01-02                 0.879671 -0.160438  0.193722  0.493397   
1   2019-01-03                 0.788898  0.059528  0.282742  0.287097   
2   2019-01-04                 0.804486 -0.156126  0.315028  0.020149   
3   2019-01-05                -0.611471 -0.195891  0.151220  0.236809   
4   2019-01-06                 0.578500 -0.082699  0.193885  0.432415   

          3         4         5         6         7  ...       758       759  \
0 -0.052131  0.442112 -0.464578 -0.171742  0.526781  ... -0.065640  0.428596   
1  0.139335  0.282810 -0.360812 -0.109893  0.553889  ... -0.324254  0.238413   
2  0.245867  0.231768 -0.288947 -0.028763  0.415299  ... -0.233439  0.249639   
3  0.021907  0.064496 -0.296357 -0.419981 -0.108710  ... -0.189058  0.247177   
4  0.034823  0.388871 -0.509751 -0.200984  0.452561  ... -0.349684  0.240410   

        760       761       762       763       764       765       766  \
0 -0.

In [ ]:
# Guardar el DataFrame con los datos agregados diarios en un archivo CSV
daily_aggregated_df.to_csv("tesla_sentiment.csv")